In [ ]:
# Código 8.0:  Bibliotecas e programas %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#                 rode sempre este código antes dos demais
import pandas as pd
import numpy as np
import time
from sklearn.tree import DecisionTreeClassifier, plot_tree
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
! pip install xgboost lime scikit-learn  > /dev/null
import xgboost as xgb
from sklearn.metrics import log_loss, roc_curve, auc,accuracy_score
import shap

In [ ]:
# Código 8.1: Criando o dataframe kim %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Primeiro baixe o arquivo Kimsport6 do github e salve em seu computador
# faça o upload do arquivo no colab
kim=pd.read_excel('/content/sample_data/Kimsport6.xlsx')
display(kim.head(3))

In [ ]:
#  Código 8.2: Árvore de classificação simples %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#===============================================================================
X= kim.drop('STATUS_MP', axis=1)
print(X.columns)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
#===============================================================================
# Criar e treinar a árvore de decisão
arvore = DecisionTreeClassifier(max_depth=2) # 2 pois é apenas ilustração
arvore.fit(X, y)
#===============================================================================
# Visualizar a árvore
plt.figure(figsize=(10, 6))  # Configurar o tamanho da imagem
class_names = [str(name) for name in y.STATUS_MP.unique()]
plot_tree(arvore, feature_names=X.columns, class_names=class_names,
          filled=False,impurity=False,proportion=True, precision=1)
plt.show()

In [ ]:
# Código 8.3: Random Forest utilizando bootsrtapping (default) %%%%%%%%%%%%%%%%%
kim=pd.read_excel('/content/sample_data/Kimsport6.xlsx')
display(kim.head(3))
X= kim.drop('STATUS_MP', axis=1)
print(X.columns)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)
#===============================================================================
start = time.time()  # Marca o início do processamento
rf = RandomForestClassifier(random_state=18,
                            min_samples_split=100,
                            min_samples_leaf=50,
                            bootstrap=True,
                            oob_score=True)  # Usar amostras OOB para validação
# grid de valores para otimização - critério AUROC
param_grid = {
    'n_estimators': [100, 200, 300], # número de rodadas
    'max_depth': [10, 15, 25], # profundidade da árvore / qto maior, maior o risco de overfitting
    'max_features': ['sqrt', 'log2']} #quantidade de vars a serem selecionadas

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=-1, verbose=2)
#n_jobs=-1: Usará todos os núcleos de CPU disponíveis no seu computador para executar a busca
#o critério de avaliação é especificado pelo parâmetro scoring. Eu prefiro AUROC
grid_search.fit(X_train, y_train)

# Mostrar os resultados de AUC para cada combinação de parâmetros
# grid_search.cv_results_ contém os resultados de todas as combinações
results = grid_search.cv_results_
# Exibir os valores de AUC para cada combinação de hiperparâmetros
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"AUC: {mean_score:.4f} | Hiperparâmetros: {params}")

print("\n")
print("Melhores parâmetros encontrados:", grid_search.best_params_)
best_rf_model = grid_search.best_estimator_
#===============================================================================
# previsões com amostra teste
y_pred = best_rf_model.predict(X_test) #classificação prevista PC=0,5 é default
y_pred_prob = best_rf_model.predict_proba(X_test)[:, 1] # probMP que é class=1
accuracy = accuracy_score(y_test, y_pred)
print("\n")
print(f'Acurácia no conjunto de teste para melhor modelo \n: {accuracy:.4f}')
fpr, tpr, thresholds = roc_curve(y_test.iloc[:, 0], y_pred_prob)
auc_score = auc(fpr, tpr)
print("\n")
print(f'AUC no conjunto de teste para melhor modelo \n: {auc_score:.4f}')
print("\n")
#===============================================================================
# importância de cada variável (critério interno de RandomForestClassifier)

importances = best_rf_model.feature_importances_
# Ordenar as importâncias em ordem decrescente
indices = np.argsort(importances)[::-1]
# Visualizar as 10 variáveis mais importantes
plt.figure(figsize=(10, 6))
plt.title("Importância das Variáveis")
plt.barh(range(X_train.shape[1]), importances[indices], align="center")
plt.yticks(range(X_train.shape[1]), X_train.columns[indices])
plt.xlabel("Importância")
plt.show()
#calculo do tempo de processamento
end = time.time()  # Marca o fim do RF
print(f"\n\nTempo de execução: {end - start:.4f} segundos")
#==============================================================================
# gerando arquivo com colunas de X_test e as probs previstas ==> kim_final_RF
kim_final_RF=X_test.copy()
kim_final_RF["status_mp"]=y_test
kim_final_RF['y_pred_prob'] = y_pred_prob
# Exibindo o DataFrame atualizado
print(kim_final_RF.head(5))

!pip install openpyxl > /dev/null
kim_final_RF.to_excel('kim_final_RF.xlsx', index=False)
from google.colab import files
files.download('kim_final_RF.xlsx')

In [ ]:
# Código 8.4: XGBOOST & e importância das variáveis com SHAP #%%%%%%%%%%%%%%%%%%%

#==============================================================================
kim=pd.read_excel('/content/sample_data/Kimsport6.xlsx')
display(kim.head(3))

X= kim.drop('STATUS_MP', axis=1)
print(X.columns)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
# Sseparando amostras
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)
#===============================================================================
start = time.time()  # Marca o início do processamento
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4,7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [10,20]
}
model = xgb.XGBClassifier(objective='binary:logistic',
                          eval_metric='logloss',early_stopping_rounds=10,learning_rate=0.1,
                          use_label_encoder=False)
#aqui, por exemplo, selecionamos logloss para medir performance
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_log_loss',
    cv=3,  # Cross-validation com 3 folds
    verbose=1,
    n_jobs=-1)

grid_search.fit(X_train, y_train,  eval_set=[(X_test, y_test)], verbose=False)
best_model = grid_search.best_estimator_

# Para acessar o número de iterações antes de ocorrer o early stop:
aux=best_model.get_booster().best_iteration
print(f'Número de árvores necessarias (early stop):  {aux}')
#==============================================================================
# Predição e cálculo do LogLoss
y_pred_prob = best_model.predict_proba(X_test)[:, 1]
log_loss_value = log_loss(y_test, y_pred_prob)
print(f'\nLogLoss no conjunto de validação: {log_loss_value:.4f}')

fpr, tpr, thresholds = roc_curve(y_test.iloc[:, 0], y_pred_prob)
auc_score = auc(fpr, tpr)
print("\n")
print(f'AUC no conjunto de teste para melhor modelo: {auc_score:.4f}')
print("\n")
print("Melhores parâmetros encontrados:", grid_search.best_params_)
end = time.time()  # Marca o fim do xgb
print(f"\n\nTempo de execução: {end - start:.4f} segundos")
#==============================================================================
# SHAP
explainer = shap.Explainer(best_model)
shap_values = explainer(X_test)

# Plot da importância das variáveis com SHAP
fig, ax = plt.subplots(figsize=(5, 5))
shap.plots.bar(shap_values, max_display=15, show=False)
bars = ax.patches  # List of the bars
# Change the color of each bar
for bar in bars:
    bar.set_facecolor('gray')  # cor das barras
for text in plt.gca().texts:  #apaga os rótulos
    text.set_visible(False)
plt.title("Importância das preditoras", fontsize=12, fontweight="bold")  # Título do gráfico
plt.xlabel("average absolute values of the SHAP values on all the training instances \n mean|SHAPvalues|", fontsize=8)  # Nome do eixo X
plt.ylabel("", fontsize=12)       # Nome do eixo Y
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

#===============================================================================
# gerando arquivo com todas as colunas de X_test e as probabilidade previstas ==> kim_final_xgb

kim_final_xgb=X_test.copy()
kim_final_xgb["status_mp"]=y_test
kim_final_xgb['y_pred_prob'] = y_pred_prob

# Exibindo o DataFrame atualizado
print(kim_final_xgb.head(5))

!pip install openpyxl > /dev/null
kim_final_xgb.to_excel('kim_final_xgb.xlsx', index=False)
from google.colab import files
files.download('kim_final_xgb.xlsx')

In [ ]:
# Código 8.5: Importância das variáveis com medida interna do xgb (Ganho)%%%%%%
#Média do ganho que a variável trouxe para a árvore.
#Considerando as vezes em participou
fig, ax = plt.subplots(figsize=(5, 5))
xgb.plot_importance(best_model, importance_type='gain',show_values=False, ax=ax)
ax.set_ylabel('', fontsize=10)
ax.set_xlabel('Importância', fontsize=12)
ax.set_title('Importância das Variáveis (Ganho)', fontsize=14)
plt.show()
